# [IAPR 2020:][iapr2020] Lab 1 ‒  Image segmentation

**Authors:** Claire Meyer, Nicolas Furrer, Philipp Schuler  
**Due date:** 26.03.2020

[iapr2018]: https://github.com/LTS5/iapr-2020

## Extract relevant data
We first need to extract the `lab-01-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
import tarfile
import os

data_base_path = os.path.join(os.pardir, "data")
data_folder = "lab-01-data"
#tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
#with tarfile.open(tar_path, mode='r:gz') as tar:
#    tar.extractall(path=data_base_path)

## Part 1: Brain segmentation

Your goal: compute the size of the brain (in pixels) in a 2D image of a human head taken by Magnetic Resonance Imaging (MRI).
* Try as many methods as you can, the more the better.
* At least region growing and contour detection.

### 1.1 Brain image visualization

In [ ]:
import skimage.io
import matplotlib.pyplot as plt
%matplotlib inline

import cv2 as cv
import numpy as np

# Load image with OpenCV
data_path = os.path.join(data_base_path, data_folder)
original_img = cv.imread(os.path.join(data_path, "brain-slice40.tiff"),0)
height, width = original_img.shape[:2]

# Plot image
M = 1
N = 1
size = 6
fig, ax = plt.subplots(N,M,figsize=(M*size,N*size))
ax.imshow(original_img, cmap="gray")
ax.set_title("MRI brain image ({} px, {} px)".format(height,width))
ax.axis("off")

#End of cell
print("Done")

### 1.2 Region growing
Add your implementation and discussion

In [ ]:
def region_growing(image, seed, low_threshold, high_threshold):
    mask = np.zeros((height+2,width+2),np.uint8)
    cv.floodFill(image, mask, seed, 255,low_threshold, high_threshold)
    return image, mask
    


brain_im_c = original_img.copy()
seed = (150,150)
image, mask = region_growing(brain_im_c, seed, 6, 6)
mask_h, mask_w = mask.shape
count = 0
for x in range(mask_w):
    for y in range(mask_h):
        if mask[x,y]==1:
            count = count + 1


#Plot
print("Seed : " , seed)
print("Brain size : ", count, " Pixels")
ax = plt.subplot(1,3,1)
ax.set_title("Base image")
ax.imshow(original_img, cmap='gray')
ax.axis('off')
ax = plt.subplot(1,3,2)
ax.set_title("Brain detection")
ax.imshow(image, cmap='gray')
ax.axis('off')
ax = plt.subplot(1,3,3,)
ax.set_title("Brain mask")
ax.imshow(mask, cmap ='gray')
ax.axis('off')


#### Discussion
Cest la la discussion

### 1.3 Contour detection
<!-- Add your implementation and discussion -->

In order to compute the size of the brain using contour detection, we start from the original image, $I_1$, and compute the subsequent images :
- $I_2$, by applying the Canny algorithm on $I_1$, the original image. We obtain the contours of the original image.
- $I_3$, by dilating $I_2$, the contours. We obtained closed contours.
- $I_4$, by flood-filling $I_3$, the closed contours, with a initial seed in the brain. We obtain a compound black image, with the closed dilated contours and the brain surface in white.
- $I_5$, by performing the "NOT" bitwise operation on $I_4$. We obtain a compound white image, with the closed dilated contours and the brain surface in black.
- $I_6$, by performing the "OR" bitwise operation on $I_3$ and $I_5$. Since $I_3$ contains the closed dilated contours, and $I_5$ anything but the contours and the brain surface, we obtain a white image with the brain surface in black, except the edges of the brain, which are white because of the previous dilation.
- $I_7$, by eroding $I_6$. We obtain a white image with the brain in black, but this time the correct surface.
- $I_8$, by performing the "NOT" bitwise operation on $I_7$. We obtain a mask of the brain surface.

From there, we just count how many pixels are white in $I_8$, the mask, to have the size of the brain in pixels. To verify the quality of the result, we also compute $I_9$, by superimposing $I_8$, the mask of the brain, on $I_1$, the original image. It allows us to assess that the brain surface is indeed correct.

<br>

All the steps from the original image to the brain mask and the quality control image (from $I_1$ to $I_9$) are displayed below, along with the implementation :

In [ ]:
# Canny detection 
t1 = 85
t2 = 200
edges_img = cv.Canny(image=original_img, threshold1=t1, threshold2=t2, apertureSize=3)

# Dilation of contours
kernel = np.zeros((3,3), np.uint8)
cv.circle(img=kernel, center=(1,1), radius=1, color=255, thickness=-1) 
dilated_edges_img = cv.dilate(edges_img, kernel, iterations=1)

# Flood-filling from pixel (150, 150)
mask = np.zeros((height+2, width+2), np.uint8)
flooded_img = dilated_edges_img.copy()
cv.floodFill(flooded_img, mask, (150,150), 255);

# Inversion of the flood filled image 
inverted_flooded_img = cv.bitwise_not(flooded_img)

# Combination of the thresholded image with the inverted flood filled image using bitwise OR operation 
dilated_segmented_img = cv.bitwise_or(dilated_edges_img, inverted_flooded_img)
segmented_img = cv.erode(dilated_segmented_img, kernel, iterations=1)

# Brain mask
brain_mask = cv.bitwise_not(segmented_img)

# Superimposition
empty_mask = np.zeros((height, width), np.uint8)
color_brain_mask = cv.merge(mv=(empty_mask, empty_mask, brain_mask))
color_original_img = cv.cvtColor(original_img, code=cv.COLOR_GRAY2RGB)
superimposed_img = cv.add(src1=color_brain_mask, src2=color_original_img)

# Number of pixels
n = np.sum(brain == 255)
print(f"{n} white pixels in the {height}x{width} image.")
print(f"Percentage of white pixels in the image : {n*100/(width*height):.2f}%")

# Plots
M = 3
N = 3
fig, ax = plt.subplots(N, M, figsize=(M*size, N*size))

ax[0,0].imshow(original_img, cmap = 'gray')
ax[0,0].set_title("$I_1$ : Original image")
ax[0,1].imshow(edges_img, cmap = 'gray')
ax[0,1].set_title("$I_2$ : Contours of original image")
ax[0,2].imshow(dilated_edges_img, cmap = 'gray')
ax[0,2].set_title("$I_3$ : Closed, dilated contours")

ax[1,0].imshow(flooded_img, cmap = 'gray')
ax[1,0].set_title("$I_4$ : Closed, dilated contours and brain surface in white")
ax[1,1].imshow(inverted_flooded_img, cmap = 'gray')
ax[1,1].set_title("$I_5$ : Closed, dilated contours and brain surface in black")
ax[1,2].imshow(dilated_segmented_img, cmap = 'gray')
ax[1,2].set_title("$I_6$ : Mask of everything but brain surface")

ax[2,0].imshow(segmented_img, cmap = 'gray')
ax[2,0].set_title("$I_7$ : Mask of brain surface, but smaller")
ax[2,1].imshow(brain_mask, cmap = 'gray')
ax[2,1].set_title("$I_8$ : Mask of brain surface")
ax[2,2].imshow(superimposed_img, cmap = 'gray')
ax[2,2].set_title("$I_9$ : Control image (mask on original image)")

for a in ax.flatten():
    a.axis("off")

#End of cell
print("Done")

The result indicates that the surface of the brain is of 15742 px, which corresponds to 24.02% of the image size, as can be seen just above. $I_9$, the quality control image, also shows that the segmentation is good, and thus that the result can be trusted.

<br>

This method requires a few meta parameters :
- $t_1$ and $t_2$, the thresholds for the Canny edges detector. Those two threshold mark the strong and weak edges : anything above the highest threshold is a strong edge, anything below the lowest threshold is deleted, and the rest is a weak edge, whose validity is determined by its link, or lack thereof, to a strong edge. Those values are determined empirically, and influence drastically the result of the edge detection. Here, we chose $t_1$=85 and $t_2$=200.
- The dilation kernel $K$ is also quite important. If too small, the dilation doesn't close the edges. If too big, some edges that should remain separate become fused. Here, the kernel is a circle of radius 1, which is equivalent to a cross on a 3x3 matrix. To ensure some degree of consistency, the same kernel is used for the subsequent erosion.
- Similarly, the dilation iteration number, if too high, can fuse edges that should remain separate. Here, we only iterated the dilation once; and thus we only eroded once as well, for consistency.
- The flood-fill necessitates an initial seed, to begin the the flood-filling from. In order to find the surface of the brain, that seed needs to be located somewhere within that area. Thus, it can take many values, as long as the coordinates are within the surface of the brain. Here, they were chosen by estimation with the naked eye (trial and error), and ($x$,$y$)=(150,150) is such a coordinate.

<br>

Initially, we tried an histogram equalization on the original image, to see if we could determine the edges more acurately that way. It proved to be severely unhelpful, to the point of being deleterious to the result. When looking at the result of an histogram equalization, we can easily see why. Indeed a lot of noise is enhanced by the equalization, to the point where there are way too many false positive when trying to detect the edges with the Canny algorithm. Below are shown the original image, the histogram equalization of that original image, and the edges detected with the Canny algorithm from that equalized image :

In [ ]:
# Histogram equalization 
equalized_img = cv.equalizeHist(original_img)

# Canny detection 
t1 = 240
t2 = 245
edges_img_2 = cv.Canny(image=equalized_img,threshold1=t1,threshold2=t2,apertureSize=3)

# Plots
M = 3
N = 1
fig, ax = plt.subplots(N, M, figsize=(M*size, N*size))

ax[0].imshow(original_img, cmap = 'gray')
ax[0].set_title("Original image")
ax[1].imshow(equalized_img, cmap = 'gray')
ax[1].set_title("Equalized original image")
ax[2].imshow(edges_img_2, cmap = 'gray')
ax[2].set_title("Contours detected on equalized image")

for a in ax.flatten():
    a.axis("off")

#End of cell
print("Done")

In order to try and detect only the strongest edges, the Canny thresholds are set very high, to keep only the strongest of edges, but that is still way too many edges; there are a lot of false positive.

### 1.4 Additional method(s)
Add your implementation and discussion

## Part 2: Shape/color segmentation

You will find hereafter three pictures taken under three different illuminations, containing some shapes with different colors. We ask you to create a routine to:

1. Count the number of shapes of each color.
2. Compute the total area (in pixels) of each color.

Please note that one specific challenge is to be robust to illumination changes. Therefore some kind of intensity normalization should probably be used.

**Note:** the routine(s) that you will write for this exercise will be useful for the final project as well, so pay special attention to it.

### 2.1 Visualization

In [ ]:
# Load images
im_names = ['arena-shapes-01', 'arena-shapes-02', 'arena-shapes-03']
filenames = [os.path.join(data_path, name) + '.png' for name in im_names]
ic = skimage.io.imread_collection(filenames)
images = skimage.io.concatenate_images(ic)
print('Number of images: ', images.shape[0])
print('Image size: {}, {} '.format(images.shape[1], images.shape[2]))
print('Number of color channels: ', images.shape[-1])

In [ ]:
# Plot images
fig, axes = plt.subplots(1, 3, figsize=(12, 12))
for ax, im, nm in zip(axes.ravel(), images, im_names):
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(nm)
plt.show()

### 2.2 Number of shapes of each color
Add your implementation and discussion

### 2.3 Total area (in pixels) of each color
Add your implementation and discussion

In [ ]:
print(f"area 1 = {stats[0][cv.CC_STAT_AREA]}")
print(f"area 2 = {stats[1][cv.CC_STAT_AREA]}")
print(f"proportion = {stats[0][cv.CC_STAT_AREA]/stats[1][cv.CC_STAT_AREA]}")

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

dt = 0.01
t = np.arange(0, 30, dt)
nse1 = np.random.randn(len(t))                 # white noise 1
nse2 = np.random.randn(len(t))                 # white noise 2

# Two signals with a coherent part at 10Hz and a random part
s1 = np.sin(2 * np.pi * 10 * t) + nse1
s2 = np.sin(2 * np.pi * 10 * t) + nse2

fig, axs = plt.subplots(2, 1)
axs[0].plot(t, s1, t, s2)
axs[0].set_xlim(0, 2)
axs[0].set_xlabel('time')
axs[0].set_ylabel('s1 and s2')
axs[0].grid(True)

cxy, f = axs[1].cohere(s1, s2, 256, 1. / dt)
axs[1].set_ylabel('coherence')

fig.tight_layout()
plt.show()
